In [274]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

In [275]:
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


In [276]:
def pluralize_gift(gift):
  """
Takes a singular noun (gift) and returns its plural form, handling words ending in 'y' or containing 'oo' or 'ee'.
  """

  if "oo" in gift:
    gift = gift.replace("oo", "ee")
  elif gift.endswith("y"):
    gift = gift[:-1] + "ies"
  else:
    gift = gift + "s"

  return gift


In [277]:
# Should work
pluralize_gift("goose")



'geese'

In [278]:

def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Builds a gift phrase for each day of the song.
    Replaces missing words with blanks, pluralizes gifts after day one,
    uses 'a' or 'an' on the first day, and joins everything into one line.
    """

   
    ## Step 1: Replace NAs with blank strings
    if pd.isna(verb): verb = ""
    if pd.isna(adjective): adjective = ""
    if pd.isna(location): location = ""
    if pd.isna(item): item = ""
    if pd.isna(num_word): num_word = ""


    ## Step 2: If the day number is larger than 1, the gift items need pluralized!
    if num > 1 and item:
        item = pluralize_gift(item)   

    ## Step 3: Figure out if a gift item starts with a vowel
    starts_vowel = item[:1].lower() in "aeiou"

    ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap) (chatgpt help)
    leading = ("an" if starts_vowel else "a") if num == 1 else num_word

    ##  Step 5: Put all of the pieces together into one string and return! ( chat gpt help)
    phrase = leading + " " + adjective + " " + item + " " + verb + " " + location
    phrase = " ".join(phrase.split())
    return phrase



In [279]:
print(make_phrase(10,"ten", "lord", "a-leaping", "",""))
print(make_phrase(1,"one","partridge","","","in a pear tree"))

ten lords a-leaping
a partridge in a pear tree


In [280]:
num_words = {
    1:"one", 2:"two", 3:"three", 4:"four", 5:"five", 6:"six",
    7:"seven", 8:"eight", 9:"nine", 10:"ten", 11:"eleven", 12:"twelve"
}

xmas["num_word"] = xmas["Day"].map(num_words)


In [281]:
xmas["Full.Phrase"] = xmas.apply(
    lambda r: make_phrase(
        num=r["Day"],
        num_word=r["num_word"],
        item=r["Gift.Item"],
        verb=r["Verb"],
        adjective=r["Adjective"],
        location=r["Location"]
    ),
    axis=1
)


In [282]:
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,num_word,Full.Phrase
0,1,first,partridge,NaN,NaN,in a pear tree,one,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two,two turtle doves
2,3,third,hen,NaN,french,NaN,three,three french hens
3,4,fourth,bird,NaN,calling,NaN,four,four calling birds
4,5,fifth,ring,NaN,golden,NaN,five,five golden rings
5,6,sixth,goose,a-laying,NaN,NaN,six,six geese a-laying
6,7,seventh,swan,a-swimming,NaN,NaN,seven,seven swans a-swimming
7,8,eighth,maid,a-milking,NaN,NaN,eight,eight maids a-milking
8,9,ninth,lady,dancing,NaN,NaN,nine,nine ladies dancing
9,10,tenth,lord,a-leaping,NaN,NaN,ten,ten lords a-leaping


In [283]:
def sing_day(dataset, num, phrase_col):
  """
  Return the lyrics of 'The Twelve Days of Christmas' for a certain day chosen.
  """
  #setup
  words = dataset.set_index("Day")["Day.in.Words"]
  phrases = dataset.set_index("Day")[phrase_col]
  verses = []
  for day in range(1, num + 1):

    # Step 1: Setup the intro line
    num_word = words.at[day]  # convert "1" to "first" etc.
    intro = "On the " + num_word + " day of Christmas my true love sent to me"

    # Step 2: Sing the gift phrases
    # (chatgpt help with part below)
    gifts = ""
    for i in range(day, 0, -1):
      line = str(phrases.at[i]).strip().rstrip(".")
      if i == 1:
        gifts = gifts + (("And " if day > 1 else "") + line + ".")
      else:
        gifts = gifts + line + ",\n"

    # Step 3: Put it all together and collect this verse
    verses.append(intro + "\n" + gifts)

  return "\n\n".join(verses)
         



In [284]:
print(sing_day(xmas, 3, "Full.Phrase"))

On the first day of Christmas my true love sent to me
a partridge in a pear tree.

On the second day of Christmas my true love sent to me
two turtle doves,
And a partridge in a pear tree.

On the third day of Christmas my true love sent to me
three french hens,
two turtle doves,
And a partridge in a pear tree.


In [285]:
print(sing_day(xmas, 12, 'Full.Phrase'))

On the first day of Christmas my true love sent to me
a partridge in a pear tree.

On the second day of Christmas my true love sent to me
two turtle doves,
And a partridge in a pear tree.

On the third day of Christmas my true love sent to me
three french hens,
two turtle doves,
And a partridge in a pear tree.

On the fourth day of Christmas my true love sent to me
four calling birds,
three french hens,
two turtle doves,
And a partridge in a pear tree.

On the fifth day of Christmas my true love sent to me
five golden rings,
four calling birds,
three french hens,
two turtle doves,
And a partridge in a pear tree.

On the sixth day of Christmas my true love sent to me
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
And a partridge in a pear tree.

On the seventh day of Christmas my true love sent to me
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
And a partridge in a

In [286]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [287]:
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN
5,6,sixth,grader,grading,NaN,NaN
6,7,seventh,senior,stressing,NaN,NaN
7,8,eighth,mom,a-calling,NaN,NaN
8,9,ninth,party,bumping,NaN,NaN
9,10,tenth,load,of laundry,NaN,NaN


In [288]:
num_words = {
    1:"one", 2:"two", 3:"three", 4:"four", 5:"five", 6:"six",
    7:"seven", 8:"eight", 9:"nine", 10:"ten", 11:"eleven", 12:"twelve"
}

xmas2["num_word"] = xmas2["Day"].map(num_words)

In [289]:
xmas2["Full.Phrase"] = xmas2.apply(
    lambda r: make_phrase(
        num=r["Day"],
        num_word=r["num_word"],
        item=r["Gift.Item"],
        verb=r["Verb"],
        adjective=r["Adjective"],
        location=r["Location"]
    ),
    axis=1
)

In [290]:
print(sing_day(xmas2, 12, 'Full.Phrase'))

On the first day of Christmas my true love sent to me
an email from Cal Poly.

On the second day of Christmas my true love sent to me
two meal points,
And an email from Cal Poly.

On the third day of Christmas my true love sent to me
three lost pens,
two meal points,
And an email from Cal Poly.

On the fourth day of Christmas my true love sent to me
four course reviews,
three lost pens,
two meal points,
And an email from Cal Poly.

On the fifth day of Christmas my true love sent to me
five practice exams,
four course reviews,
three lost pens,
two meal points,
And an email from Cal Poly.

On the sixth day of Christmas my true love sent to me
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
And an email from Cal Poly.

On the seventh day of Christmas my true love sent to me
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
And an email from Cal Poly.

On the eighth day 